![title](sql_logo.PNG)

When it comes to data storage, the word "database" often comes to mind,
reason being that data are easily manage in a database. There are many
engines we can use to work with databases in order to effectively performed 
tasks such as storing, updating, deleting and querying data in a database.

**Two major category:**
    
    * Structural Query Language (SQL)
    * Non Structural Query Language (NoSQL)
    
**SQL databases:** They are table based database and have pre-defined schema.
    
**NoSQl databases:** They are document based with key-value paired format of storing data (JSON).
                They have dynamic schemas for unstructured data.
                
***In this notebook we will briefly go over some syntax in SQL (Mysql) and NoSql (Elasticsearch).***

### Note!!
Before we get started we first need to run the sql server for MYsql.<br>
in our case we are using Xampp, from Xampp we turned on Mysql server.<br>
We can now make connection with the sql server from Python environment.

In [3]:
# Import sqlalchemy 
from sqlalchemy import create_engine

## Create connection to the SQL database

In [4]:
try:
    # Create an engine to the census database
    engine = create_engine("mysql+pymysql://'root':@localhost:3306/first database")
    connection = engine.connect()
    # Print the table names
    print('connceted to mysql database')
except:
    print('not connected!')



connceted to mysql database


### View  list of tables in the connected database

In [5]:
engine.table_names()

['customer_order', 'don_rn', 'ukconsumption']

### Select Table (ukconsumption) and View  schema 

In [6]:
from sqlalchemy import Table, MetaData

In [8]:
metadata = MetaData()
table_data = Table('ukconsumption',metadata, autoload = True,autoload_with = engine)

print(repr(table_data))

Table('ukconsumption', MetaData(bind=None), Column('Country', VARCHAR(length=9), table=<ukconsumption>), Column('AlcDrinks', INTEGER(display_width=3), table=<ukconsumption>), Column('Beverages', INTEGER(display_width=2), table=<ukconsumption>), Column('CarcaseMeat', INTEGER(display_width=3), table=<ukconsumption>), Column('Cereals', INTEGER(display_width=4), table=<ukconsumption>), Column('Cheese', INTEGER(display_width=3), table=<ukconsumption>), Column('Confectionery', INTEGER(display_width=2), table=<ukconsumption>), Column('FatsOils', INTEGER(display_width=3), table=<ukconsumption>), Column('Fish', INTEGER(display_width=3), table=<ukconsumption>), Column('FreshFruit', INTEGER(display_width=4), table=<ukconsumption>), Column('FreshPot', INTEGER(display_width=4), table=<ukconsumption>), Column('FreshVeg', INTEGER(display_width=3), table=<ukconsumption>), Column('OtherMeat', INTEGER(display_width=3), table=<ukconsumption>), Column('OtherVeg', INTEGER(display_width=3), table=<ukconsump

### Simple SQL query

In [9]:
query = 'SELECT * FROM ukconsumption'
sql_object = connection.execute(query)
sql_object.fetchall()

[('England', 375, 57, 245, 1472, 105, 54, 193, 147, 1102, 720, 253, 685, 488, 198, 360, 1374, 156),
 ('N.Ireland', 135, 47, 267, 1494, 66, 41, 209, 93, 674, 1033, 143, 586, 355, 187, 334, 1506, 139),
 ('Scotland', 458, 53, 242, 1462, 103, 62, 184, 122, 957, 566, 171, 750, 418, 220, 337, 1572, 139),
 ('Wales', 475, 73, 227, 1582, 103, 64, 235, 160, 1137, 874, 265, 803, 570, 203, 365, 1256, 175)]

![title](elasticsearch_logo.PNG)

### Note!!
Before we get started we first need to run the ElasticSearch server in our local device.<br>
We can now make connection with the ElasticSearch server from Python environment.

## Create index

In [10]:
from datetime import datetime
from elasticsearch import Elasticsearch

# by default we connect to localhost:9200
es = Elasticsearch()

# create an index in elasticsearch, ignore status code 400 (index already exists)
es.indices.create(index='my-index', ignore=400)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'my-index'}

## Create index (test-index) and input a single record (doc)

In [95]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}
res = es.index(index="test-index", id=1, body=doc)
print(res['result'])

created


## Retrieve data elasticsearch's index (test-index)

In [96]:
res = es.get(index="test-index", id=1)
print(res['_source'])

{'author': 'kimchy', 'text': 'Elasticsearch: cool. bonsai cool.', 'timestamp': '2020-09-03T21:46:16.944326'}


## Create data in form of json or dictionary (documents)

In [3]:
patients = {
         "Name":{"0":"John","1":"Nick","2":"Ali","3":"Joseph"},
         "Gender":{"0":"Male","1":"Male","2":"Female","3":"Male"},
         "Nationality":{"0":"UK","1":"French","2":"USA","3":"Brazil"},
         "Age" :{"0":10,"1":25,"2":35,"3":29}
}

### View the json data in a Pandas table

In [186]:
import pandas as pd
data = pd.DataFrame(patients)
data

,Name,Gender,Nationality,Age
0,John,Male,UK,10
1,Nick,Male,French,25
2,Ali,Female,USA,35
3,Joseph,Male,Brazil,29


### Store documents into one id.

In [13]:
actions =[
    {
        '_index': 'my-index',
        '_type': 'product',
        '_id': 2,
        '_source': patients
}
]

bulk(es,actions)

(1, [])

### Store one document into one id.

In [14]:
actions =[
    {
        '_index': 'my-index',
        '_type': 'product',
        '_id': 4,
        '_source': {"name":"victor"}
}
]

bulk(es,actions)

(1, [])

## Create data to load into ElasticSearch

In [163]:
 cars = [
        {"car_Name":"Honda", "Price": 10000, "Model":2005, "Power": 1300},
        {"car_Name":"Toyota", "Price": 12000, "Model":2010, "Power": 1600},
        {"car_Name":"Audi", "Price": 25000, "Model":2017, "Power": 1800},
        {"car_Name":"Ford", "Price": 28000, "Model":2009, "Power": 1200},

    ]

### Load documents  through generator

In [179]:
def format_doc(json_list):
    
    for i,j in enumerate(json_list):
        temp =  {
            '_index': 'my-index',
            '_type': '_doc',
            '_id': i+10,
            '_source': j
        }
        yield temp

bulk(es,format_doc(cars))

(4, [])

### Load documents through Iterable 

In [180]:
actions =[
    {
        '_index': 'my-index',
        '_type': '_doc',
        '_id': i+10,
        '_source': j
}
    
for i,j in enumerate(cars)]

#actions = iter(actions)
bulk(es,actions)

(4, [])

# Advance Elasticsearch (insert, delete and query) 
#Interraction with Pandas dataframe

## Create index (House_Innovation)

In [3]:
import pandas as pd
from datetime import datetime
from elasticsearch import Elasticsearch

# by default we connect to localhost:9200
es = Elasticsearch()

# create an index in elasticsearch, ignore status code 400 (index already exists)
es.indices.create(index='house_innovation', ignore=400)

{'error': {'root_cause': [{'type': 'resource_already_exists_exception',
    'reason': 'index [house_innovation/CxZq1B-1SGK700Q9PudAhA] already exists',
    'index_uuid': 'CxZq1B-1SGK700Q9PudAhA',
    'index': 'house_innovation'}],
  'type': 'resource_already_exists_exception',
  'reason': 'index [house_innovation/CxZq1B-1SGK700Q9PudAhA] already exists',
  'index_uuid': 'CxZq1B-1SGK700Q9PudAhA',
  'index': 'house_innovation'},
 'status': 400}

### Function to iterate pandas table records

In [273]:
def data_to_doc(dataframe):
     # dataframe row iterator (index,series of column : value)
    for indx, col_val_map in dataframe.iterrows():
        yield indx, col_val_map.to_dict() #convert to dictionary

### Read data from pandas, pass it through the function (data_to_doc func)
### and finally to ElasticSearch.

In [274]:
actions = []

file_path = 'DATA/PurchaseData.csv'
data = pd.read_csv(file_path, chunksize= 10050) #read data in chunks

for df in data: # read each chunk
    for i , j in data_to_doc(df): #iterate through data generator (index,dict)
      
        action = {
            '_index' : 'house_innovation',     #format the data for elasticsearch
            '_type'  : '_doc',
            '_id'    : i,
            '_source': j    
        }
        actions.append(action)   # add each action to a list 
        
    print('True')
    
    bulk(es,actions)   # commit multiple input data to elasticsearch
    

True


C:\Users\VICTOR\anaconda3\envs\elasticsearch\lib\site-packages\elasticsearch\connection\base.py:177: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


True


### How to deleted index

In [271]:
es.indices.delete(index='house_innovation')

{'acknowledged': True}

### How to query ElasticSearch Engine

In [94]:
query_body = {'query':{'bool': {'must': {'match':{'ShopCode': 1}},
              'filter': [{'match':{'Amount':800}},{'match':{'Month_To_Japan':2}}]
                      }}}

In [92]:
query_body = {'query': {'bool':{'must': [{'match':{'Amount':800}},{'match':{'Month_To_Japan':2}}]}}}

### Search for match in  ElasticSearch engine and return result into Python environment

In [95]:
res = es.search(body=query_body, index='house_innovation',doc_type='_doc')

In [96]:
res['hits']

{'total': {'value': 7, 'relation': 'eq'},
 'max_score': 1.0,
 'hits': [{'_index': 'house_innovation',
   '_type': '_doc',
   '_id': '58',
   '_score': 1.0,
   '_source': {'ID': 'KR90107',
    'Month_To_Japan': 2,
    'ShopCode': 1,
    'ItemCode': 3,
    'Amount': 800}},
  {'_index': 'house_innovation',
   '_type': '_doc',
   '_id': '59',
   '_score': 1.0,
   '_source': {'ID': 'KR90107',
    'Month_To_Japan': 2,
    'ShopCode': 1,
    'ItemCode': 3,
    'Amount': 800}},
  {'_index': 'house_innovation',
   '_type': '_doc',
   '_id': '65',
   '_score': 1.0,
   '_source': {'ID': 'KR40113',
    'Month_To_Japan': 2,
    'ShopCode': 1,
    'ItemCode': 3,
    'Amount': 800}},
  {'_index': 'house_innovation',
   '_type': '_doc',
   '_id': '71',
   '_score': 1.0,
   '_source': {'ID': 'KR90123',
    'Month_To_Japan': 2,
    'ShopCode': 1,
    'ItemCode': 3,
    'Amount': 800}},
  {'_index': 'house_innovation',
   '_type': '_doc',
   '_id': '75',
   '_score': 1.0,
   '_source': {'ID': 'KR70125',
